# Export CLSS Assessment Data and Import into WebEOC 
This notebook is designed to let users select the CLSS Service environment they want to evaluate (Development, Testing, or Production), then identify the desired layer or table to export(currently limited to "Assessments" table) and export the results to a CSV file based on the WebEOC Lifeline Board format. 
This notebook also supports reading Assessment data from CLSS and importing data into WebEOC using WebEOC's API.

Requirements:
- User must have access to ArcGIS Online with permissions to execute ArcGIS Notebooks ***OR*** ability to run Jupyter Notebooks on local computer.
- A WebEOC account with access to update board records.

Limitations:
- CLSS data contains status at the Lifeline level but not Component Level.
- No ability to associate assessment data from CLSS with the related incident in WebEOC - this is hard coded. 
- No connection established between Hazards related to Assessment data with Event Type in WebEOC Incident.
- Note for Juvare team: Current Lifeline board (Situation Update Board) contain an attribute for Lifeline status; however the Lifeline R/Y/G/U status represented on the dashboard only takes into account the 'most impacted' of it's constituent component data.  Recommend allowing LL status representation in dashboard to visualize the status tracked in the LL field. 

Next Steps:
- add capability to push CLSS assessment records to feature service that can be incorporated into ArcGIS Dashboard, etc. (summary of lifleine by jurisdiction)
- add option to select which WebEOC incident to push new CLSS Assessment record, use most recent incident as default if nothing selected
- add support to push new incident tracked in CLSS > WebEOC

Note: used copilot liberally to assist creation of functions

Sections:
[Export CLSS data > WebEOC Lifeline Board with API](###export-clss-data--webeoc-lifeline-board-with-api)

#### Import requirements, define global variables

In [1]:
# Import the required libraries
from arcgis.gis import GIS
import json
import requests
import pandas as pd
from pandas import json_normalize
import ipywidgets as widgets
from datetime import datetime

In [ ]:
# Set parameters for WebEOC Access
base_url = 'https://trial1.demo.webeocasp.com/trial1/api/rest.svc'
# Set Default Incident and Position
position = "CMD Incident Commander"
incident = "Trial"

# Get username and password from config file 
with open('config.json') as config_file:
    config = json.load(config_file)

# AGO_username = config['AGO_username']
# AGO_password = config['AGO_password']
webeoc_username = config['webeoc_username']
webeoc_password = config['webeoc_password']

# Get the current date-time
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Translate hexcodes to readable colors
colorTranslate = {
    "#C52038": "Red",
    "#FBBA16": "Yellow",
    "#5E9C42": "Green",
    "#919395": "Unknown",
    "NONE": "NONE"
}

# These are the CLSS ArcGIS online Service Endpoints 
clssServices = [('None', 'None'),  # Example ID for Dev
             ('CLSS Dev', '4cec9a93384543e0a676e3ad892362bb'),  
             ('CLSS Test', 'f726c24e77d5442c9a9f456eec62ae5d'),  
             ('CLSS Demo', 'c5aea531e01d49358b73123e334b4c0a')]

# This is the list of WebEOC fields and matching CLSS fields
WebEocCLSS_Translation = [
    ("component_911 dispatch_details", "component_911_and_dispatch_details"),
    ("component_911 dispatch_status", "component_911_and_dispatch_status"),
    ("component_911 dispatch_included", "component_911_and_dispatch_included"),
    ("component_agriculture_details", "component_agriculture_details"),
    ("component_agriculture_status", "component_agriculture_status"),
    ("component_agriculture_included", "component_agriculture_included"),
    ("component_alerts, warnings, messages_details", "component_alerts_warnings_messages_details"),
    ("component_alerts, warnings, messages_status", "component_alerts_warnings_messages_status"),
    ("component_alerts, warnings, messages_included", "component_alerts_warnings_messages_included"),
    ("component_aviation_details", "component_aviation_details"),
    ("component_aviation_status", "component_aviation_status"),
    ("component_aviation_included", "component_aviation_included"),
    ("component_community safety_details", "component_community_safety_details"),
    ("component_community safety_status", "component_community_safety_status"),
    ("component_community safety_included", "component_community_safety_included"),
    ("component_facilities_details", "component_facilities_details"),
    ("component_facilities_status", "component_facilities_status"),
    ("component_facilities_included", "component_facilities_included"),
    ("component_fatality management_details", "component_fatality_management_details"),
    ("component_fatality management_status", "component_fatality_management_status"),
    ("component_fatality management_included", "component_fatality_management_included"),
    ("component_finance_details", "component_financial_services_details"),
    ("component_finance_status", "component_financial_services_status"),
    ("component_finance_included", "component_financial_services_included"),
    ("component_fire service_details", "component_fire_services_details"),
    ("component_fire service_status", "component_fire_services_status"),
    ("component_fire service_included", "component_fire_services_included"),
    ("component_food_details", "component_food_details"),
    ("component_food_status", "component_food_status"),
    ("component_food_included", "component_food_included"),
    ("component_fuel_details", "component_fuel_details"),
    ("component_fuel_status", "component_fuel_status"),
    ("component_fuel_included", "component_fuel_included"),
    ("component_government service_details", "component_govt_services_details"),
    ("component_government service_status", "component_govt_services_status"),
    ("component_government service_included", "component_govt_services_included"),
    ("component_hazmat pollutants contaminants_details", "component_hazmat_pollutants_contaminants_details"),
    ("component_hazmat pollutants contaminants_status", "component_hazmat_pollutants_contaminants_status"),
    ("component_hazmat pollutants contaminants_included", "component_hazmat_pollutants_contaminants_included"),
    ("component_highway roadway_details", "component_highway_roads_details"),
    ("component_highway roadway_status", "component_highway_roads_status"),
    ("component_highway roadway_included", "component_highway_roads_included"),
    ("component_hydration_details", "component_hydration_details"),
    ("component_hydration_status", "component_hydration_status"),
    ("component_hydration_included", "component_hydration_included"),
    ("component_infrastructure_details", "component_infrastructure_details"),
    ("component_infrastructure_status", "component_infrastructure_status"),
    ("component_infrastructure_included", "component_infrastructure_included"),
    ("component_law enforcement security_details", "component_law_enforcement_details"),
    ("component_law enforcement security_status", "component_law_enforcement_status"),
    ("component_law enforcement security_included", "component_law_enforcement_included"),
    ("component_maritime_details", "component_maritime_details"),
    ("component_maritime_status", "component_maritime_status"),
    ("component_maritime_included", "component_maritime_included"),
    ("component_mass transit_details", "component_mass_transit_details"),
    ("component_mass transit_status", "component_mass_transit_status"),
    ("component_mass transit_included", "component_mass_transit_included"),
    ("component_medical care_details", "component_medical_care_details"),
    ("component_medical care_status", "component_medical_care_status"),
    ("component_medical care_included", "component_medical_care_included"),
    ("component_medical supply chain_details", "component_medical_supply_details"),
    ("component_medical supply chain_status", "component_medical_supply_status"),
    ("component_medical supply chain_included", "component_medical_supply_included"),
    ("component_patient movement_details", "component_patient_movement_details"),
    ("component_patient movement_status", "component_patient_movement_status"),
    ("component_patient movement_included", "component_patient_movement_included"),
    ("component_drinking water infrastructure_details", "component_potable_water_details"),
    ("component_drinking water infrastructure_status", "component_potable_water_status"),
    ("component_drinking water infrastructure_included", "component_potable_water_included"),
    ("component_power grid_details", "component_power_details"),
    ("component_power grid_status", "component_power_status"),
    ("component_power grid_included", "component_power_included"),
    ("component_public health_details", "component_public_health_details"),
    ("component_public health_status", "component_public_health_status"),
    ("component_public health_included", "component_public_health_included"),
    ("component_railway_details", "component_railway_details"),
    ("component_railway_status", "component_railway_status"),
    ("component_railway_included", "component_railway_included"),
    ("component_responder communications_details", "component_responder_comms_details"),
    ("component_responder communications_status", "component_responder_comms_status"),
    ("component_responder communications_included", "component_responder_comms_included"),
    ("component_search rescue_details", "component_search_and_rescue_details"),
    ("component_search rescue_status", "component_search_and_rescue_status"),
    ("component_search rescue_included", "component_search_and_rescue_included"),
    ("component_shelter_details", "component_shelter_details"),
    ("component_shelter_status", "component_shelter_status"),
    ("component_shelter_included", "component_shelter_included"),
    ("component_wastewater infrastructure_details", "component_waste_water_details"),
    ("component_wastewater infrastructure_status", "component_waste_water_status"),
    ("component_wastewater infrastructure_included", "component_waste_water_included"),
    ("lifeline_communications_status", "lifeline_communications_status"),
    ("lifeline_communications_included", "lifeline_communications_included"),
    ("lifeline_energy_status", "lifeline_energy_status"),
    ("lifeline_energy_included", "lifeline_energy_included"),
    ("lifeline_food, hydration, shelter_status", "lifeline_food_status"),
    ("lifeline_food, hydration, shelter_included", "lifeline_food_included"),
    ("lifeline_hazardous materials_status", "lifeline_hazmat_status"),
    ("lifeline_hazardous materials_included", "lifeline_hazmat_included"),
    ("lifeline_health & medical_status", "lifeline_health_status"),
    ("lifeline_health & medical_included", "lifeline_health_included"),
    ("lifeline_safety & security_status", "lifeline_safety_status"),
    ("lifeline_safety & security_included", "lifeline_safety_included"),
    ("lifeline_transportation_status", "lifeline_transportation_status"),
    ("lifeline_transportation_included", "lifeline_transportation_included"),
    ("lifeline_water systems_status", "lifeline_water_status"),
    ("lifeline_water systems_included", "lifeline_water_included")
]

# create global variable for Assessment data and df_content dataframe
global dfAssessment
df_content = pd.DataFrame()

In [ ]:
# Sign into the ArcGIS Online or Enterprise Portal
gis = GIS("home")
# gis = GIS("https://ghis.maps.arcgis.com/", AGO_username, AGO_password)

#### Define Functions

In [4]:
########################################################
def content_to_dataframe(record):
    """
    Convert the 'content' field of a selected record to a dataframe.
    
    Parameters:
    record (dict): The selected record containing the 'content' field.
    
    Returns:
    pd.DataFrame: A dataframe containing the flattened JSON data.
    """
    # Extract the 'content' field and load it as JSON
    content_json = json.loads(record['Content'])
    
    # Normalize the JSON data to create a new dataframe
    df_content = json_normalize(content_json)
    # print(df_content.head())
    
    # Add the Assessment Name to the dataframe
    df_content['AssessmentName'] = record['Name']
    
    # Add a new column with the text-based color that translates to the hexcode 'color' field
    df_content['status'] = df_content['color'].map(colorTranslate)

    return df_content

########################################################

def get_component_status_info(componentStatus):
    """
    Extract component status information from the content field and return it as a human-readable delimited string.
    
    Parameters:
    componentStatus (list): The component status list containing JSON data.
    
    Returns:
    str: A human-readable delimited string of component status information.
    """

    # Ensure component_statuses is a dictionary and has 'indicators' key
    if isinstance(component_statuses, dict) and 'indicators' in component_statuses:
        indicators = component_statuses['indicators']
        return "".join([f"{indicator['indicator']}: {indicator['status']}\n\n" for indicator in indicators])
    return "No indicators available"


#### Get CLSS Lifeline Feature Service

In [ ]:
# Global variables
selected_feature_service = None
selected_assessment = None

def handle_service_selection(service_id):
    """Handles feature service selection and directly processes the Assessment table."""
    global selected_feature_service
    selected_feature_service = gis.content.get(service_id)
    print(f"Selected Feature Service: {selected_feature_service.title}")

    # Automatically find and process the Assessment table
    process_assessment_table(selected_feature_service)

def process_assessment_table(feature_service):
    """Finds the Assessment table and processes it."""
    tables = feature_service.tables
    assessment_table = next(
        (table for table in tables if table.properties.name == "Assessment"), None
    )

    if not assessment_table:
        print("Assessment table not found in the selected feature service.")
        return

    # print(f"Processing Assessment Table: {assessment_table.properties.name}")
    try:
        # Query all features from the Assessment table
        assessment_features = assessment_table.query(where="1=1", out_fields="*").features
        df_assessment = pd.DataFrame([feature.attributes for feature in assessment_features])

        # Parse CreationDate into a human-readable format
        if 'CreationDate' in df_assessment.columns:
            try:
                df_assessment['CreationDate'] = pd.to_datetime(
                    df_assessment['CreationDate'], unit='ms'
                ).dt.strftime('%m-%d-%Y')
            except Exception as e:
                print(f"Error parsing CreationDate: {e}")
                df_assessment['CreationDate'] = "Invalid Date"
        else:
            df_assessment['CreationDate'] = "Date Not Available"

        # Present a dropdown to select an assessment
        present_assessment_choices(df_assessment)

    except Exception as e:
        print(f"Error processing Assessment Table: {e}")

def present_assessment_choices(df_assessment):
    """Displays a dropdown for selecting an assessment."""
    assessment_options = [
        (f"{row['Name']} - {row['CreationDate']}", row['OBJECTID'])
        for _, row in df_assessment.iterrows()
    ]

    assessment_choice = widgets.Dropdown(
        options=assessment_options,
        description="Select Assessment:",
        disabled=False,
    )

    def handle_assessment_selection(object_id):
        """Handles assessment selection and sets the global selected assessment."""
        global selected_assessment
        selected_assessment = df_assessment[df_assessment['OBJECTID'] == object_id].iloc[0]
        print(f"Selected Assessment: {selected_assessment['Name']} - {selected_assessment['CreationDate']}")

    # Attach handler and display the widget
    display(assessment_choice)
    assessment_choice.observe(lambda change: handle_assessment_selection(change["new"]), names="value")

# Main widget for selecting the feature service
service_choice = widgets.Dropdown(
    options=clssServices,
    description="Select Feature Service:",
    disabled=False,
)

# Attach handler and display the widget
display(service_choice)
service_choice.observe(lambda change: handle_service_selection(change["new"]), names="value")


Dropdown(description='Select Feature Service:', options=(('None', 'None'), ('CLSS Dev', '4cec9a93384543e0a676e…

Selected Feature Service: CLSS_FeatureService_Dev


Dropdown(description='Select Assessment:', options=(('Aca Test Assessment 1 - 08-07-2024', 1), ('Pg Test Asses…

Selected Assessment: 2024-09_wildfire - Parcel 5 Part 2 - 09-05-2024
Selected Assessment: Component Status Test Assessment - 11-19-2024


##### Query the selected feature service to get the Assessment table


In [77]:
df_assessment_content = content_to_dataframe(selected_assessment)

def add_prefix(df, column, prefix, suffix=None):
    if suffix:
        return df[column].apply(lambda x: f"{prefix}_{x}_{suffix}")
    return df[column].apply(lambda x: f"{prefix}_{x}")

# Extract fields from componentStatuses
def safe_get(d, keys, default='Unknown'):
    """
    Safely get a nested value from a dictionary or return a default if any level is missing.
    """
    if not isinstance(d, dict):
        return default
    value = d
    for key in keys:
        value = value.get(key, default) if isinstance(value, dict) else default
    return value

# Function to clean redundant column names
def clean_column_name(column):
    if "details_componentdetails" in column:
        return column.replace("details_componentdetails", "details")
    # elif "details_componentscore" in column:
    #     return column.replace("details_componentscore", "score")
    elif "details_componentstatus" in column:
        return column.replace("details_componentstatus", "status")
    return column

# Function to extract indicator details
def extract_component_details(component_statuses):
    indicators = component_statuses.get('indictors', [])
    # print ("".join([f"{indicator['indicator']}: {indicator['status']}\n\n" for indicator in indicators]))
    return "".join([f"{indicator['indicator']}: {indicator['status']}\n\n" for indicator in indicators])

# Extract fields from componentStatuses with safeguards
df_exploded_components = df_assessment_content.explode('componentStatuses')
df_exploded_components = df_exploded_components.assign(
    componentName=df_exploded_components['componentStatuses'].apply(lambda x: safe_get(x, ['componentName'], 'Unknown Component')),
    componentDetails=df_exploded_components['componentStatuses'].apply(extract_component_details),
    componentStatus=df_exploded_components['componentStatuses'].apply(lambda x: safe_get(x, ['status', 'colorAsText'], 'Unknown')),
    # componentScore=df_exploded_components['componentStatuses'].apply(lambda x: safe_get(x, ['status', 'score'], 'Unknown')),
    lifelineStatus=df_exploded_components['status']  # Assign the existing 'status' column directly
)

# Add prefixes
df_exploded_components['lifelineName'] = add_prefix(df_exploded_components, 'lifelineName', 'lifeline', 'status')
df_exploded_components['componentName'] = add_prefix(df_exploded_components, 'componentName', 'component', 'details')

# Select relevant fields
df_selected = df_exploded_components[['AssessmentName', 'lifelineName', 'lifelineStatus', 'componentName', 'componentStatus', 'componentDetails']]

# Handle duplicates by aggregation
df_selected = df_selected.groupby(['AssessmentName', 'lifelineName'], as_index=False).agg({
    'lifelineStatus': 'first',
    'componentName': 'first',
    'componentStatus': 'first',
    'componentDetails': 'first'
    # 'componentScore': 'first'
})

# Pivot lifeline statuses
df_lifeline_pivot = df_selected.pivot(index='AssessmentName', columns='lifelineName', values='lifelineStatus')

# Pivot component statuses, scores, and details
df_component_pivot = (
    df_selected.pivot(index='AssessmentName', columns='componentName', values=['componentStatus', 'componentDetails'])
    .swaplevel(axis=1)
    .sort_index(axis=1)
)
# Flatten and clean multi-level columns in df_component_pivot
df_component_pivot.columns = [
    clean_column_name(f"{col[0]}_{col[1]}".lower() if isinstance(col, tuple) else col.lower())
    for col in df_component_pivot.columns
]

# Combine lifeline and component data, normalize column names
df_combined = df_lifeline_pivot.join(df_component_pivot)
df_combined.columns = ['_'.join(col).lower() if isinstance(col, tuple) else col.lower() for col in df_combined.columns]
# Fill NaN values with 'Unknown'
df_combined = df_combined.fillna('Unknown')

# Rename columns based on WebEocCLSS_Translation
translation_dict = dict(WebEocCLSS_Translation)
df_combined.rename(columns=translation_dict, inplace=True)

# Display the resulting dataframe
print('Finished processing assessment dataframe')
# df_combined.head()

Finished processing assessment dataframe


#### Export to CSV

In [79]:
# define assessment name 
assessment_name = df_combined.index[0]

# Create the filename
filename = f"exports/{assessment_name}_{current_datetime}.csv"

# Export the df_content DataFrame to a CSV file with the new filename
df_combined.to_csv(filename, index=False)

--------------------------------------

### Export CLSS data > WebEOC Lifeline Board with API

reference for WebEOC API: https://ghinternational-my.sharepoint.com/my?id=%2Fpersonal%2Feendrulat%5Fghinternational%5Fcom%2FDocuments%2FMicrosoft%20Teams%20Chat%20Files%2FWebEOC%20REST%20API%20Technical%20Guide%20Sep2022%2Epdf&parent=%2Fpersonal%2Feendrulat%5Fghinternational%5Fcom%2FDocuments%2FMicrosoft%20Teams%20Chat%20Files


In [81]:
# Create the session to be utilized by the REST API.
def create_session(base_url):
	url = f"{base_url}/sessions"

	payload = json.dumps({
		"username": webeoc_username,
		"password": webeoc_password,
		"position": position,
		"incident": incident
	})
	headers = {
		'Content-Type': 'application/json',
	}
	session = requests.Session()
	session.post(url, headers=headers, data=payload)

	return session

# End the session.
def close_session(session):
  url = f"{base_url}/sessions"
  session.delete(url)

In [82]:

def get_current_incidents(base_url, session):
    url = f"{base_url}/incidents"
    response = session.get(url)
    
    if response.status_code == 200:
        incidents = response.json()
        return incidents
    else:
        response.raise_for_status()

# Create a session
session = create_session(base_url)

# Get the list of current incidents
try:
    incidents = get_current_incidents(base_url, session)
    print("Current Incidents:", incidents)
except requests.exceptions.RequestException as e:
    print("Failed to retrieve incidents:", e)

# Close the session
close_session(session)

# Create a dropdown widget for selecting an incident
incident_options = [('Select Incident', None)] + [(incident, incident) for incident in incidents]
incident_choice = widgets.Dropdown(
    options=incident_options,
    description='Incident:',
    disabled=False,
)

# Display the dropdown widget
display(incident_choice)

# Function to handle the selection
def on_incident_selected(change):
    incident = change['new']
    if incident:
        print(f"Selected Incident: {incident}")
        # update session
        session = create_session(base_url)

# Attach the handler to the dropdown widget
incident_choice.observe(on_incident_selected, names='value')



Current Incidents: ['2023-10-18 Fire', 'Trial']


Dropdown(description='Incident:', options=(('Select Incident', None), ('2023-10-18 Fire', '2023-10-18 Fire'), …

Selected Incident: Trial


In [83]:
# Create a POST Request to create a record using the provided details.
def create_board_record(base_url, session, board_name, inputviewname, record_data):
	create_record_url = f"{base_url}/board/{board_name}/input/{inputviewname}"
	print (create_record_url)
	payload = {'data': json.dumps(record_data)}

	response = session.post(create_record_url, json=payload)

	if response.status_code == 201:
		# Record created successfully, return the response
		return response.json()
	else:
		# Record creation failed, raise an exception with the error message
		response.raise_for_status()

In [ ]:

# Define the timestamp
timeStamp = f'Updated by WebEOC API as {position}' + datetime.now().strftime("%m-%d-%Y %H:%M:%S")

# Convert the DataFrame to a dictionary
df_dict = df_combined.to_dict(orient='index')

# Extract the first (and only) assessment
assessment_data = df_dict[assessment_name]

# Add the timestamp to each component and lifeline
for key in list(assessment_data.keys()):  # Iterate over a list of the dictionary keys
    if key.startswith('component_') or key.startswith('lifeline_'):
        assessment_data[f"{key}_timestamp"] = timeStamp

# Convert the dictionary to a JSON object
# assessment_json = json.dumps(assessment_data, indent=4)
record_data = assessment_data

# Print the JSON object
print(assessment_data)

{'lifeline_communications_status': 'Yellow', 'lifeline_energy_status': 'Yellow', 'lifeline_food_status': 'Yellow', 'lifeline_hazmat_status': 'Yellow', 'lifeline_health_status': 'Yellow', 'lifeline_safety_status': 'Green', 'lifeline_transportation_status': 'Green', 'lifeline_water_status': 'Yellow', 'component_911_and_dispatch_details': 'Damage to Public Safety Answering Point (PSAP) Infrastructure: Minimal / No Impact\n\nDelays to Dispatch: Moderate Impact\n\nPublic Safety Answering Points: Significant Impact\n\n', 'component_911_and_dispatch_status': 'Yellow', 'component_agriculture_details': 'Agricultural Contamination or Disease: Significant Impact\n\nCapacity for Livestock Evacuation: Moderate Impact\n\nCapacity for livestock Mass Burial/De-Population: Moderate Impact\n\nFood for Livestock beyond normal conditions: Moderate Impact\n\nShelter for Livestock beyond normal conditions: Significant Impact\n\nWater Source for Livestock beyond normal conditions: Unknown\n\n', 'component_ag

In [ ]:
def main(board_name, view_name, record_data):

        session = create_session(base_url)

        try:
                response = create_board_record(base_url, session, board_name, view_name, record_data)
                print("Record created successfully:", response)
        except requests.exceptions.RequestException as e:
                print("Failed to create record:", e)

        close_session(session)


###### Example usage - Event Reporting: Sig Event

board_name = 'Situation Report - Working'
view_name = 'Lifeline_components_all'

main(board_name, view_name, record_data)

https://trial1.demo.webeocasp.com/trial1/api/rest.svc/board/Situation Report - Working/input/Lifeline_components_all
Record created successfully: None


In [29]:
# note: there was no input view for all lifelines/components 
# created new input view: Lifeline_components_all

In [86]:
record_data

{'lifeline_communications_status': 'Yellow',
 'lifeline_energy_status': 'Yellow',
 'lifeline_food_status': 'Yellow',
 'lifeline_hazmat_status': 'Yellow',
 'lifeline_health_status': 'Yellow',
 'lifeline_safety_status': 'Green',
 'lifeline_transportation_status': 'Green',
 'lifeline_water_status': 'Yellow',
 'component_911_and_dispatch_details': 'Damage to Public Safety Answering Point (PSAP) Infrastructure: Minimal / No Impact\n\nDelays to Dispatch: Moderate Impact\n\nPublic Safety Answering Points: Significant Impact\n\n',
 'component_911_and_dispatch_status': 'Yellow',
 'component_agriculture_details': 'Agricultural Contamination or Disease: Significant Impact\n\nCapacity for Livestock Evacuation: Moderate Impact\n\nCapacity for livestock Mass Burial/De-Population: Moderate Impact\n\nFood for Livestock beyond normal conditions: Moderate Impact\n\nShelter for Livestock beyond normal conditions: Significant Impact\n\nWater Source for Livestock beyond normal conditions: Unknown\n\n',
 'c